In [1]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

2025-02-17 12:26:47.538868: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-17 12:26:47.568988: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-17 12:26:47.569027: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-17 12:26:47.569050: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-17 12:26:47.575075: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-17 12:26:47.582314: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
file  = open('data.txt', 'r', encoding = "utf8")

In [3]:
lines=[]
for i in file:
    lines.append(i)

In [4]:
data=""
for i in lines:
    data = ' '.join(lines)

In [5]:
 data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')

In [6]:
data = data.split()
data = ' '.join(data)

In [7]:
data[:1000]

"THE ADVENTURES OF SHERLOCK HOLMES Arthur Conan Doyle Table of contents A Scandal in Bohemia The Red-Headed League A Case of Identity The Boscombe Valley Mystery The Five Orange Pips The Man with the Twisted Lip The Adventure of the Blue Carbuncle The Adventure of the Speckled Band The Adventure of the Engineer's Thumb The Adventure of the Noble Bachelor The Adventure of the Beryl Coronet The Adventure of the Copper Beeches A SCANDAL IN BOHEMIA Table of contents Chapter 1 Chapter 2 Chapter 3 CHAPTER I To Sherlock Holmes she is always the woman. I have seldom heard him mention her under any other name. In his eyes she eclipses and predominates the whole of her sex. It was not that he felt any emotion akin to love for Irene Adler. All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. He was, I take it, the most perfect reasoning and observing machine that the world has seen, but as a lover he would have placed himself in a false positio

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

In [9]:
pickle.dump(tokenizer, open('token.pkl', 'wb'))

In [10]:
sequence_data = tokenizer.texts_to_sequences([data])[0]

In [11]:
sequence_data[:10]

[1, 1561, 5, 129, 34, 647, 4498, 4499, 226, 5]

In [12]:
len(sequence_data)

105879

In [13]:
vocab_size = len(tokenizer.word_index)+1

In [14]:
vocab_size

8200

In [15]:
sequence=[]
for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequence.append(words)

In [16]:
len(sequence)

105876

In [17]:
sequence = np.array(sequence)

In [18]:
sequence

array([[   1, 1561,    5,  129],
       [1561,    5,  129,   34],
       [   5,  129,   34,  647],
       ...,
       [  28,    1, 8198, 8199],
       [   1, 8198, 8199, 3187],
       [8198, 8199, 3187, 3186]])

In [19]:
X=[]
y=[]

for i in sequence:
    X.append(i[0:3])
    y.append(i[3])

In [20]:
X=np.array(X)
y=np.array(y)

In [21]:
y = to_categorical(y, num_classes=vocab_size)

In [22]:
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

Building the LSTM Model

In [23]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length = 3))
model.add(LSTM(1000, return_sequences = True))
model.add(LSTM(1000))
model.add(Dense(1000, activation='relu'))
model.add(Dense(vocab_size, activation="softmax"))

2025-02-17 12:27:42.137831: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             82000     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 8200)              8208200   
                                                                 
Total params: 21339200 (81.40 MB)
Trainable params: 21339200 (81.40 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("next_words.h5", monitor="loss", verbose = 1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer = Adam(learning_rate = 0.001))
model.fit(X, y, epochs=2, batch_size=64, callbacks=[checkpoint])

Epoch 1/2
 178/1655 [==>...........................] - ETA: 3:13 - loss: 6.9673

KeyboardInterrupt: 

In [26]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available: 0


In [29]:
print(torch.cuda.is_available())

True


In [28]:
import torch

In [30]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Mon_Apr__3_17:16:06_PDT_2023
Cuda compilation tools, release 12.1, V12.1.105
Build cuda_12.1.r12.1/compiler.32688072_0


In [31]:
!nvidia-smi

Mon Feb 17 12:34:25 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.230.02             Driver Version: 535.230.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:35:00.0 Off |                    0 |
| N/A   33C    P8              16W /  72W |      3MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [32]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPUs detected:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.14.1
GPUs detected: []


In [33]:
!ls /usr/local/cuda/lib64 | grep cudnn

In [34]:
!pip install nvidia-cudnn-cu12
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/lib64


[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import tensorflow as tf
print(tf.sysconfig.get_build_info())

2025-02-17 12:45:06.233734: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-17 12:45:06.391652: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-17 12:45:06.391724: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-17 12:45:06.392289: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-17 12:45:06.470498: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-17 12:45:06.479941: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow[and-cuda]

Found existing installation: tensorflow 2.14.1
Uninstalling tensorflow-2.14.1:
  Would remove:
    /system/conda/miniconda3/envs/cloudspace/bin/estimator_ckpt_converter
    /system/conda/miniconda3/envs/cloudspace/bin/import_pb_to_tensorboard
    /system/conda/miniconda3/envs/cloudspace/bin/saved_model_cli
    /system/conda/miniconda3/envs/cloudspace/bin/tensorboard
    /system/conda/miniconda3/envs/cloudspace/bin/tf_upgrade_v2
    /system/conda/miniconda3/envs/cloudspace/bin/tflite_convert
    /system/conda/miniconda3/envs/cloudspace/bin/toco
    /system/conda/miniconda3/envs/cloudspace/bin/toco_from_protos
    /system/conda/miniconda3/envs/cloudspace/lib/python3.10/site-packages/tensorflow-2.14.1.dist-info/*
    /system/conda/miniconda3/envs/cloudspace/lib/python3.10/site-packages/tensorflow/*
Proceed (Y/n)? 

In [ ]:
Y